# BestTradingBot Demo Run

This notebook demonstrates how to run a backtest and analyze the results using the BTB library directly from Python rather than the command-line interface.

In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Add project root to path for imports
sys.path.append(os.path.abspath(".."))

from btb.backtest.engine import Backtester
from btb.utils.config import load_config

## 1. Load Configuration

In [ ]:
# Load backtest configuration
config = load_config("../config/backtest_config.yaml")
print("Configuration loaded:")
print(f"Symbol(s): {config['backtest']['symbols']}")
print(f"Timeframe(s): {config['backtest']['timeframes']}")
print(f"Strategy: {config['backtest']['strategy']}")
print(f"Date range: {config['backtest']['start_date']} to {config['backtest']['end_date']}")

## 2. Run Backtest

In [ ]:
# Create strategy parameters
strategy_params = {
    "short_period": 10,
    "long_period": 30,
    "stop_loss": 0.02,  # 2%
    "take_profit": 0.05,  # 5%
    "position_size": 0.2,  # 20% of capital per trade
}

# Add strategy parameters to config
config["strategy_params"] = strategy_params

# Initialize backtester
backtester = Backtester(config)

# Run backtest
print("Running backtest...")
results = backtester.run()

## 3. Analyze Results

In [ ]:
# Display performance metrics from the results dictionary
metrics = results.get("metrics", {})
print("\nPerformance Metrics:")
print(f"Initial Capital: ${metrics.get('initial_capital', 'N/A'):.2f}")
print(f"Final Capital: ${metrics.get('final_capital', 'N/A'):.2f}")
print(f"Total Return: {metrics.get('total_return_pct', 'N/A'):.2f}%")
print(f"Sharpe Ratio: {metrics.get('sharpe_ratio', 'N/A'):.2f}")
print(f"Max Drawdown: {metrics.get('max_drawdown_pct', 'N/A'):.2f}%")
print(f"Win Rate: {metrics.get('win_rate', 0) * 100:.2f}%")
print(f"Total Trades: {metrics.get('total_trades', 'N/A')}")
print(f"Profit Factor: {metrics.get('profit_factor', 'N/A'):.2f}")
print(f"Avg Profit (%): {metrics.get('avg_profit_pct', 0)*100:.2f}%")
print(f"Avg Loss (%): {metrics.get('avg_loss_pct', 0)*100:.2f}%")

In [ ]:
# Plot equity curve
plt.figure(figsize=(12, 6))
plt.plot(results["equity_curve"])
plt.title("Equity Curve")
plt.xlabel("Trading Days")
plt.ylabel("Portfolio Value ($)")
plt.grid(True)
plt.show()

# Drawdown is already calculated and printed in the metrics summary above.
# The equity curve plot provides a visual representation of performance over time.

## 4. Analyze Trades

In [ ]:
# Analyze trades
trades = results["trades"]
if trades:
    trades_df = pd.DataFrame(trades)

    # Calculate trade durations
    trades_df["entry_time"] = pd.to_datetime(trades_df["entry_time"])
    trades_df["exit_time"] = pd.to_datetime(trades_df["exit_time"])
    trades_df["duration"] = trades_df["exit_time"] - trades_df["entry_time"]
    trades_df["duration_hours"] = trades_df["duration"].dt.total_seconds() / 3600

    # Display trade summary (using pre-calculated metrics where available)
    print("\nTrade Summary:")
    print(f"Total Trades: {metrics.get('total_trades', len(trades_df))}") # Use metric if available, else count df rows
    print(f"Winning Trades: {metrics.get('winning_trades', len(trades_df[trades_df['profit'] > 0]))}")
    print(f"Losing Trades: {metrics.get('losing_trades', len(trades_df[trades_df['profit'] <= 0]))}")
    print(f"Average Profit: ${metrics.get('avg_profit', trades_df['profit'][trades_df['profit'] > 0].mean()):.2f}")
    print(f"Average Profit %: {metrics.get('avg_profit_pct', trades_df['profit_pct'][trades_df['profit_pct'] > 0].mean())*100:.2f}%")
    print(f"Average Loss: ${metrics.get('avg_loss', trades_df['profit'][trades_df['profit'] <= 0].mean()):.2f}")
    print(f"Average Loss %: {metrics.get('avg_loss_pct', trades_df['profit_pct'][trades_df['profit_pct'] <= 0].mean())*100:.2f}%")
    print(f"Average Trade Duration: {trades_df['duration_hours'].mean():.2f} hours") # Duration not in standard metrics

    # Show first few trades
    trades_df.head()

In [ ]:
# Plot profit distribution
if len(trades_df) > 0:
    plt.figure(figsize=(12, 6))
    sns.histplot(trades_df["profit_pct"], bins=30, kde=True)
    plt.axvline(x=0, color="red", linestyle="--")
    plt.title("Trade Profit Distribution (%)")
    plt.xlabel("Profit (%)")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

In [ ]:
# Plot trade results by exit reason
if len(trades_df) > 0:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x="exit_reason", y="profit_pct", data=trades_df)
    plt.title("Trade Results by Exit Reason")
    plt.xlabel("Exit Reason")
    plt.ylabel("Profit (%)")
    plt.grid(True, axis="y")
    plt.show()

## 5. Examine a Symbol's Performance

In [ ]:
# Select a symbol for detailed examination
symbol = config["backtest"]["symbols"][0]
timeframe = config["backtest"]["timeframes"][0]
symbol_key = f"{symbol}_{timeframe}"

if symbol_key in results["symbol_results"]:
    symbol_result = results["symbol_results"][symbol_key]

    # Plot equity curve for this symbol
    plt.figure(figsize=(12, 6))
    plt.plot(symbol_result["equity_curve"])
    plt.title(f"{symbol} {timeframe} Equity Curve")
    plt.xlabel("Trading Days")
    plt.ylabel("Portfolio Value ($)")
    plt.grid(True)
    plt.show()

    # Display symbol metrics
    print(f"\n{symbol} {timeframe} Performance:")
    print(f"Final Capital: ${symbol_result['final_capital']:.2f}")
    print(f"Return: {symbol_result['return'] * 100:.2f}%")

    # Filter trades for this symbol
    symbol_trades = [t for t in trades if t["symbol"] == symbol]
    print(f"Number of Trades: {len(symbol_trades)}")

## 6. Strategy Parameter Optimization (Example)

This section demonstrates a simple loop to test different parameter values. For more advanced optimization techniques like walk-forward analysis, consider using tools like `btb.backtest.walk_forward.WalkForwardAnalyzer` if applicable.

In [ ]:
def optimize_parameters(config, parameter, values):
    """Run backtest with different parameter values."""
    results = []

    for value in values:
        # Create a copy of the config
        test_config = config.copy()

        # Update the parameter
        if "strategy_params" not in test_config:
            test_config["strategy_params"] = {}
        test_config["strategy_params"][parameter] = value

        # Run backtest
        backtester = Backtester(test_config)
        backtest_results = backtester.run()

        # Store results
        results.append(
            {
                "parameter": parameter,
                "value": value,
                "total_return": backtest_results["metrics"]["total_return_pct"],
                "sharpe_ratio": backtest_results["metrics"]["sharpe_ratio"],
                "max_drawdown": backtest_results["metrics"]["max_drawdown_pct"],
                "win_rate": backtest_results["metrics"]["win_rate"] * 100,
            }
        )

    return pd.DataFrame(results)

In [ ]:
# Example optimization of moving average periods (uncomment to run)
# short_periods = [5, 10, 15, 20]
# print("Optimizing short_period parameter...")
# optimization_results = optimize_parameters(config, 'short_period', short_periods)
# optimization_results

In [ ]:
# Plot optimization results (uncomment to run)
# plt.figure(figsize=(12, 8))

# plt.subplot(2, 2, 1)
# plt.plot(optimization_results['value'], optimization_results['total_return'], marker='o')
# plt.title('Total Return vs. Parameter Value')
# plt.xlabel('Parameter Value')
# plt.ylabel('Total Return (%)')
# plt.grid(True)

# plt.subplot(2, 2, 2)
# plt.plot(optimization_results['value'], optimization_results['sharpe_ratio'], marker='o')
# plt.title('Sharpe Ratio vs. Parameter Value')
# plt.xlabel('Parameter Value')
# plt.ylabel('Sharpe Ratio')
# plt.grid(True)

# plt.subplot(2, 2, 3)
# plt.plot(optimization_results['value'], optimization_results['max_drawdown'], marker='o')
# plt.title('Max Drawdown vs. Parameter Value')
# plt.xlabel('Parameter Value')
# plt.ylabel('Max Drawdown (%)')
# plt.grid(True)

# plt.subplot(2, 2, 4)
# plt.plot(optimization_results['value'], optimization_results['win_rate'], marker='o')
# plt.title('Win Rate vs. Parameter Value')
# plt.xlabel('Parameter Value')
# plt.ylabel('Win Rate (%)')
# plt.grid(True)

# plt.tight_layout()
# plt.show()

## 7. Save Results

In [ ]:
# Save backtest results to file (uncomment to run)
# result_path = '../results/backtest_results.json'
# os.makedirs(os.path.dirname(result_path), exist_ok=True)
# backtester.save_results(result_path)
# print(f"Results saved to {result_path}")

## 8. Conclusion

In this notebook, we demonstrated how to:

1. Configure and run a backtest with the BestTradingBot library
2. Analyze backtesting results including equity curves, drawdowns, and trade statistics
3. Optimize strategy parameters for better performance
4. Save backtest results for future reference

These techniques can be applied to develop, test, and optimize your own trading strategies using the BTB framework.